In [1]:
import sys
import os

SOURCE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__name__)))
sys.path.insert(0, SOURCE_DIR)

In [2]:
import malaya_speech
import tensorflow as tf

In [3]:
import torch
from torch import nn
from torch.nn import functional as F

In [4]:
tf.compat.v1.enable_eager_execution()

In [5]:
from malaya_speech.train.model.vits import modules, transforms, model

In [6]:
x = tf.random.normal(shape = (1, 10, 192))
x_mask = tf.ones(shape = (1, 10, 1))
w = tf.random.normal(shape = (1, 10))

In [7]:
dp = model.StochasticDurationPredictor(192, 192, 3, 0.3, 4, gin_channels=0)

In [8]:
dp(x, x_mask, tf.expand_dims(w, -1), training = True)

(1, 10, 2) (1, 10, 1)
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

(10,) (10,)
(1, 10, 2) (1, 10, 1)
(10,) (10,)
(1, 10, 2) (1, 10, 1)
(10,) (10,)
(1, 10, 2) (1, 10, 1)
(10,) (10,)
(1, 10, 2) (1, 10, 1)
(10,) (10,)
(1, 10, 2) (1, 10, 1)
(3,) (3,)


InvalidArgumentError: Input to reshape is a tensor with 3 values, but the requested shape has 10 [Op:Reshape]

In [4]:
channels = 2

In [8]:
m = nn.Parameter(torch.zeros(channels,1))
m

Parameter containing:
tensor([[0.],
        [0.]], requires_grad=True)

In [10]:
m * torch.randn((1, 2, 10))

torch.Size([1, 2, 10])

In [14]:
o = 0

In [15]:
o += tf.reduce_sum(tf.zeros([1, channels]) * tf.random.normal((1, 10, 2)), axis = [1,2])

In [5]:
# modules.ConvFlow()

In [6]:
# StochasticDurationPredictor(hidden_channels, 192, 3, 0.5, 4, gin_channels=gin_channels)
# def __init__(self, in_channels, filter_channels, kernel_size, p_dropout, n_flows=4, gin_channels=0)

In [7]:
filter_channels = 192
kernel_size = 3
convflow = modules.ConvFlow(2, filter_channels, kernel_size, n_layers=3)

In [8]:
x = tf.random.normal(shape = (1, 100, 2))
x_mask = tf.ones(shape = (1, 100, 1))

In [12]:
convflow(x, x_mask, reverse=True)

<tf.Tensor: id=5157, shape=(1, 100, 2), dtype=float32, numpy=
array([[[ 1.5666072 , -2.4329512 ],
        [ 0.9565053 ,  0.7426468 ],
        [ 0.9703825 ,  0.74713105],
        [ 1.1183755 ,  0.5719689 ],
        [ 0.12016225, -1.4151428 ],
        [ 0.44691294, -1.2847147 ],
        [-0.61637676,  0.3745149 ],
        [ 0.28276357, -0.10016853],
        [-0.0167621 , -0.50790286],
        [-0.81279355, -2.5225022 ],
        [ 1.9150769 , -0.95889354],
        [ 0.3612458 , -0.7331414 ],
        [ 0.48345795,  0.6402668 ],
        [-1.2884052 ,  0.41396394],
        [-2.2541056 ,  1.1267474 ],
        [ 0.11691213,  0.61497796],
        [ 0.21372145, -1.3142843 ],
        [-0.30053446,  1.7102852 ],
        [ 0.2008264 ,  0.13372372],
        [-0.5892715 ,  0.48494962],
        [ 0.58625406,  0.72502786],
        [-0.40763742,  0.22599801],
        [-0.16812006,  0.6694594 ],
        [ 0.37198266, -0.23508203],
        [-0.86109346, -0.08036351],
        [-0.09207939,  0.1364009 ],
  

In [14]:
x1,unnormalized_widths, unnormalized_heights, unnormalized_derivatives = convflow(x, x_mask, reverse=False)

In [19]:
x1_pt = torch.from_numpy(x1.numpy())
unnormalized_widths_pt = torch.from_numpy(unnormalized_widths.numpy())
unnormalized_heights_pt = torch.from_numpy(unnormalized_heights.numpy())
unnormalized_derivatives_pt = torch.from_numpy(unnormalized_derivatives.numpy())

In [142]:
import tensorflow as tf

import numpy as np


DEFAULT_MIN_BIN_WIDTH = 1e-3
DEFAULT_MIN_BIN_HEIGHT = 1e-3
DEFAULT_MIN_DERIVATIVE = 1e-3


def piecewise_rational_quadratic_transform_tf(inputs,
                                           unnormalized_widths,
                                           unnormalized_heights,
                                           unnormalized_derivatives,
                                           inverse=False,
                                           tails=None,
                                           tail_bound=1.,
                                           min_bin_width=DEFAULT_MIN_BIN_WIDTH,
                                           min_bin_height=DEFAULT_MIN_BIN_HEIGHT,
                                           min_derivative=DEFAULT_MIN_DERIVATIVE):

    if tails is None:
        spline_fn = rational_quadratic_spline_tf
        spline_kwargs = {}
    else:
        spline_fn = unconstrained_rational_quadratic_spline_tf
        spline_kwargs = {
            'tails': tails,
            'tail_bound': tail_bound
        }

    outputs, logabsdet = spline_fn_tf(
        inputs=inputs,
        unnormalized_widths=unnormalized_widths,
        unnormalized_heights=unnormalized_heights,
        unnormalized_derivatives=unnormalized_derivatives,
        inverse=inverse,
        min_bin_width=min_bin_width,
        min_bin_height=min_bin_height,
        min_derivative=min_derivative,
        **spline_kwargs
    )
    return outputs, logabsdet


def searchsorted_tf(bin_locations, inputs, eps=1e-6):
    bin_locations = tf.concat([
        bin_locations[..., :-1],
        tf.expand_dims(bin_locations[..., -1] + 1e-6, -1)], axis=-1)
    return tf.reduce_sum(
        tf.cast(inputs[..., None] >= bin_locations, tf.int32),
        axis=-1
    ) - 1


def unconstrained_rational_quadratic_spline_tf(inputs,
                                            unnormalized_widths,
                                            unnormalized_heights,
                                            unnormalized_derivatives,
                                            inverse=False,
                                            tails='linear',
                                            tail_bound=1.,
                                            min_bin_width=DEFAULT_MIN_BIN_WIDTH,
                                            min_bin_height=DEFAULT_MIN_BIN_HEIGHT,
                                            min_derivative=DEFAULT_MIN_DERIVATIVE):
    inside_interval_mask = (inputs >= -tail_bound) & (inputs <= tail_bound)
    outside_interval_mask = ~inside_interval_mask

    outputs = tf.zeros_like(inputs)
    logabsdet = tf.zeros_like(inputs)

    if tails == 'linear':
        unnormalized_derivatives = tf.pad(unnormalized_derivatives, [(0, 0), (0, 0), (0, 0), (1, 1)])
        constant = np.log(np.exp(1 - min_derivative) - 1)

        unnormalized_derivatives = tf.concat([
            tf.expand_dims(unnormalized_derivatives[..., 0] + constant, -1),
            unnormalized_derivatives[..., 1:-1],
            tf.expand_dims(unnormalized_derivatives[..., -1] + constant, -1),
        ], axis=-1)

        # outputs[outside_interval_mask] = inputs[outside_interval_mask]
        # logabsdet[outside_interval_mask] = 0

        outputs = tf.where(outside_interval_mask, inputs, outputs)
        tiled = tf.fill(tf.shape(logabsdet), 0.0)
        logabsdet = tf.where(outside_interval_mask, tiled, logabsdet)
    else:
        raise RuntimeError(f'{tails} tails are not implemented.')

    d = tf.shape(unnormalized_widths)[-1]
    unnormalized_widths = unnormalized_widths[tf.tile(tf.expand_dims(inside_interval_mask, -1), [1, 1, 1, d])]
    unnormalized_widths = tf.reshape(unnormalized_widths, [-1, d])

    d = tf.shape(unnormalized_heights)[-1]
    unnormalized_heights = unnormalized_heights[tf.tile(tf.expand_dims(inside_interval_mask, -1), [1, 1, 1, d])]
    unnormalized_heights = tf.reshape(unnormalized_heights, [-1, d])

    d = tf.shape(unnormalized_derivatives)[-1]
    unnormalized_derivatives = unnormalized_derivatives[tf.tile(tf.expand_dims(inside_interval_mask, -1), [1, 1, 1, d])]
    unnormalized_derivatives = tf.reshape(unnormalized_derivatives, [-1, d])

    return rational_quadratic_spline_tf(
        inputs=inputs[inside_interval_mask],
        unnormalized_widths=unnormalized_widths,
        unnormalized_heights=unnormalized_heights,
        unnormalized_derivatives=unnormalized_derivatives,
        inverse=inverse,
        left=-tail_bound, right=tail_bound, bottom=-tail_bound, top=tail_bound,
        min_bin_width=min_bin_width,
        min_bin_height=min_bin_height,
        min_derivative=min_derivative
    )
    outputs_ = tf.reshape(outputs_, tf.shape(inside_interval_mask))
    logabsdet_ = tf.reshape(logabsdet_, tf.shape(inside_interval_mask))

    outputs = tf.where(inside_interval_mask, outputs_, outputs)
    logabsdet = tf.where(inside_interval_mask, logabsdet_, logabsdet)

    return outputs, logabsdet


def rational_quadratic_spline_tf(inputs,
                              unnormalized_widths,
                              unnormalized_heights,
                              unnormalized_derivatives,
                              inverse=False,
                              left=0., right=1., bottom=0., top=1.,
                              min_bin_width=DEFAULT_MIN_BIN_WIDTH,
                              min_bin_height=DEFAULT_MIN_BIN_HEIGHT,
                              min_derivative=DEFAULT_MIN_DERIVATIVE):

    num_bins = tf.shape(unnormalized_widths)[-1]

    widths = tf.nn.softmax(unnormalized_widths, axis=-1)
    widths = min_bin_width + (1 - min_bin_width * tf.cast(num_bins, tf.float32)) * widths
    cumwidths = tf.cumsum(widths, axis=-1)
    cumwidths = tf.pad(cumwidths, [[0, 0], [1, 0]])
    cumwidths = (right - left) * cumwidths + left

    # cumwidths[..., 0] = left
    # cumwidths[..., -1] = right

    cumwidths = tf.concat([
        tf.expand_dims(tf.fill(tf.shape(cumwidths[..., 0]), left), -1),
        cumwidths[..., 1:-1],
        tf.expand_dims(tf.fill(tf.shape(cumwidths[..., -1]), right), -1),
    ], axis=-1)
    widths = cumwidths[..., 1:] - cumwidths[..., :-1]

    derivatives = min_derivative + tf.math.softplus(unnormalized_derivatives)

    heights = tf.nn.softmax(unnormalized_heights, axis=-1)
    heights = min_bin_height + (1 - min_bin_height * tf.cast(num_bins, tf.float32)) * heights
    cumheights = tf.cumsum(heights, axis=-1)
    cumheights = tf.pad(cumheights, [[0, 0], [1, 0]])
    cumheights = (top - bottom) * cumheights + bottom

    # cumheights[..., 0] = bottom
    # cumheights[..., -1] = top

    cumheights = tf.concat([
        tf.expand_dims(tf.fill(tf.shape(cumheights[..., 0]), bottom), -1),
        cumheights[..., 1:-1],
        tf.expand_dims(tf.fill(tf.shape(cumheights[..., -1]), top), -1),
    ], axis=-1)

    heights = cumheights[..., 1:] - cumheights[..., :-1]

    if inverse:
        bin_idx = searchsorted_tf(cumheights, inputs)[..., None]
    else:
        bin_idx = searchsorted_tf(cumwidths, inputs)[..., None]

    input_cumwidths = tf.gather_nd(cumwidths, bin_idx, batch_dims=1)
    input_bin_widths = tf.gather_nd(widths, bin_idx, batch_dims=1)

    input_cumheights = tf.gather_nd(cumheights, bin_idx, batch_dims=1)
    delta = heights / widths
    input_delta = tf.gather_nd(delta, bin_idx, batch_dims=1)

    input_derivatives = tf.gather_nd(derivatives, bin_idx, batch_dims=1)
    input_derivatives_plus_one = tf.gather_nd(derivatives[..., 1:], bin_idx, batch_dims=1)

    input_heights = tf.gather_nd(heights, bin_idx, batch_dims=1)

    if inverse:
        a = (((inputs - input_cumheights) * (input_derivatives
                                             + input_derivatives_plus_one
                                             - 2 * input_delta)
              + input_heights * (input_delta - input_derivatives)))
        b = (input_heights * input_derivatives
             - (inputs - input_cumheights) * (input_derivatives
                                              + input_derivatives_plus_one
                                              - 2 * input_delta))
        c = - input_delta * (inputs - input_cumheights)

        discriminant = (b ** 2) - 4 * a * c

        root = (2 * c) / (-b - tf.sqrt(discriminant))
        outputs = root * input_bin_widths + input_cumwidths

        theta_one_minus_theta = root * (1 - root)
        denominator = input_delta + ((input_derivatives + input_derivatives_plus_one - 2 * input_delta)
                                     * theta_one_minus_theta)
        derivative_numerator = (input_delta ** 2) * (input_derivatives_plus_one * (root**2)
                                                     + 2 * input_delta * theta_one_minus_theta
                                                     + input_derivatives * ((1 - root)**2))
        logabsdet = tf.log(derivative_numerator) - 2 * tf.log(denominator)
        return outputs, -logabsdet
    else:
        theta = (inputs - input_cumwidths) / input_bin_widths
        theta_one_minus_theta = theta * (1 - theta)

        numerator = input_heights * (input_delta * (theta**2)
                                     + input_derivatives * theta_one_minus_theta)
        denominator = input_delta + ((input_derivatives + input_derivatives_plus_one - 2 * input_delta)
                                     * theta_one_minus_theta)
        outputs = input_cumheights + numerator / denominator

        derivative_numerator = (input_delta**2) * (input_derivatives_plus_one * (theta**2)
                                                   + 2 * input_delta * theta_one_minus_theta
                                                   + input_derivatives * ((1 - theta)**2))
        logabsdet = tf.log(derivative_numerator) - 2 * tf.log(denominator)

        return outputs, logabsdet


In [143]:
import torch
from torch.nn import functional as F

import numpy as np


DEFAULT_MIN_BIN_WIDTH = 1e-3
DEFAULT_MIN_BIN_HEIGHT = 1e-3
DEFAULT_MIN_DERIVATIVE = 1e-3


def piecewise_rational_quadratic_transform(inputs, 
                                           unnormalized_widths,
                                           unnormalized_heights,
                                           unnormalized_derivatives,
                                           inverse=False,
                                           tails=None, 
                                           tail_bound=1.,
                                           min_bin_width=DEFAULT_MIN_BIN_WIDTH,
                                           min_bin_height=DEFAULT_MIN_BIN_HEIGHT,
                                           min_derivative=DEFAULT_MIN_DERIVATIVE):

    if tails is None:
        spline_fn = rational_quadratic_spline
        spline_kwargs = {}
    else:
        spline_fn = unconstrained_rational_quadratic_spline
        spline_kwargs = {
            'tails': tails,
            'tail_bound': tail_bound
        }

    outputs, logabsdet = spline_fn(
            inputs=inputs,
            unnormalized_widths=unnormalized_widths,
            unnormalized_heights=unnormalized_heights,
            unnormalized_derivatives=unnormalized_derivatives,
            inverse=inverse,
            min_bin_width=min_bin_width,
            min_bin_height=min_bin_height,
            min_derivative=min_derivative,
            **spline_kwargs
    )
    return outputs, logabsdet


def searchsorted(bin_locations, inputs, eps=1e-6):
    bin_locations[..., -1] += eps
    return torch.sum(
        inputs[..., None] >= bin_locations,
        dim=-1
    ) - 1


def unconstrained_rational_quadratic_spline(inputs,
                                            unnormalized_widths,
                                            unnormalized_heights,
                                            unnormalized_derivatives,
                                            inverse=False,
                                            tails='linear',
                                            tail_bound=1.,
                                            min_bin_width=DEFAULT_MIN_BIN_WIDTH,
                                            min_bin_height=DEFAULT_MIN_BIN_HEIGHT,
                                            min_derivative=DEFAULT_MIN_DERIVATIVE):
    inside_interval_mask = (inputs >= -tail_bound) & (inputs <= tail_bound)
    outside_interval_mask = ~inside_interval_mask

    outputs = torch.zeros_like(inputs)
    logabsdet = torch.zeros_like(inputs)

    if tails == 'linear':
        unnormalized_derivatives = F.pad(unnormalized_derivatives, pad=(1, 1))
        constant = np.log(np.exp(1 - min_derivative) - 1)
        unnormalized_derivatives[..., 0] = constant
        unnormalized_derivatives[..., -1] = constant

        outputs[outside_interval_mask] = inputs[outside_interval_mask]
        logabsdet[outside_interval_mask] = 0
    else:
        raise RuntimeError('{} tails are not implemented.'.format(tails))

    return rational_quadratic_spline(
        inputs=inputs[inside_interval_mask],
        unnormalized_widths=unnormalized_widths[inside_interval_mask, :],
        unnormalized_heights=unnormalized_heights[inside_interval_mask, :],
        unnormalized_derivatives=unnormalized_derivatives[inside_interval_mask, :],
        inverse=inverse,
        left=-tail_bound, right=tail_bound, bottom=-tail_bound, top=tail_bound,
        min_bin_width=min_bin_width,
        min_bin_height=min_bin_height,
        min_derivative=min_derivative
    )

    return outputs, logabsdet

def rational_quadratic_spline(inputs,
                              unnormalized_widths,
                              unnormalized_heights,
                              unnormalized_derivatives,
                              inverse=False,
                              left=0., right=1., bottom=0., top=1.,
                              min_bin_width=DEFAULT_MIN_BIN_WIDTH,
                              min_bin_height=DEFAULT_MIN_BIN_HEIGHT,
                              min_derivative=DEFAULT_MIN_DERIVATIVE):
    if torch.min(inputs) < left or torch.max(inputs) > right:
        raise ValueError('Input to a transform is not within its domain')

    num_bins = unnormalized_widths.shape[-1]

    if min_bin_width * num_bins > 1.0:
        raise ValueError('Minimal bin width too large for the number of bins')
    if min_bin_height * num_bins > 1.0:
        raise ValueError('Minimal bin height too large for the number of bins')

    widths = F.softmax(unnormalized_widths, dim=-1)
    widths = min_bin_width + (1 - min_bin_width * num_bins) * widths
    cumwidths = torch.cumsum(widths, dim=-1)
    cumwidths = F.pad(cumwidths, pad=(1, 0), mode='constant', value=0.0)
    cumwidths = (right - left) * cumwidths + left
    cumwidths[..., 0] = left
    cumwidths[..., -1] = right
    widths = cumwidths[..., 1:] - cumwidths[..., :-1]

    derivatives = min_derivative + F.softplus(unnormalized_derivatives)

    heights = F.softmax(unnormalized_heights, dim=-1)
    heights = min_bin_height + (1 - min_bin_height * num_bins) * heights
    cumheights = torch.cumsum(heights, dim=-1)
    cumheights = F.pad(cumheights, pad=(1, 0), mode='constant', value=0.0)
    cumheights = (top - bottom) * cumheights + bottom
    cumheights[..., 0] = bottom
    cumheights[..., -1] = top
    heights = cumheights[..., 1:] - cumheights[..., :-1]

    if inverse:
        bin_idx = searchsorted(cumheights, inputs)[..., None]
    else:
        bin_idx = searchsorted(cumwidths, inputs)[..., None]

    input_cumwidths = cumwidths.gather(-1, bin_idx)[..., 0]
    input_bin_widths = widths.gather(-1, bin_idx)[..., 0]

    input_cumheights = cumheights.gather(-1, bin_idx)[..., 0]
    delta = heights / widths
    input_delta = delta.gather(-1, bin_idx)[..., 0]

    input_derivatives = derivatives.gather(-1, bin_idx)[..., 0]
    input_derivatives_plus_one = derivatives[..., 1:].gather(-1, bin_idx)[..., 0]

    input_heights = heights.gather(-1, bin_idx)[..., 0]

    if inverse:
        a = (((inputs - input_cumheights) * (input_derivatives
                                             + input_derivatives_plus_one
                                             - 2 * input_delta)
              + input_heights * (input_delta - input_derivatives)))
        b = (input_heights * input_derivatives
             - (inputs - input_cumheights) * (input_derivatives
                                              + input_derivatives_plus_one
                                              - 2 * input_delta))
        c = - input_delta * (inputs - input_cumheights)

        discriminant = b.pow(2) - 4 * a * c
        assert (discriminant >= 0).all()

        root = (2 * c) / (-b - torch.sqrt(discriminant))
        outputs = root * input_bin_widths + input_cumwidths

        theta_one_minus_theta = root * (1 - root)
        denominator = input_delta + ((input_derivatives + input_derivatives_plus_one - 2 * input_delta)
                                     * theta_one_minus_theta)
        derivative_numerator = input_delta.pow(2) * (input_derivatives_plus_one * root.pow(2)
                                                     + 2 * input_delta * theta_one_minus_theta
                                                     + input_derivatives * (1 - root).pow(2))
        logabsdet = torch.log(derivative_numerator) - 2 * torch.log(denominator)

        return outputs, -logabsdet
    else:
        theta = (inputs - input_cumwidths) / input_bin_widths
        theta_one_minus_theta = theta * (1 - theta)

        numerator = input_heights * (input_delta * theta.pow(2)
                                     + input_derivatives * theta_one_minus_theta)
        denominator = input_delta + ((input_derivatives + input_derivatives_plus_one - 2 * input_delta)
                                     * theta_one_minus_theta)
        outputs = input_cumheights + numerator / denominator

        derivative_numerator = input_delta.pow(2) * (input_derivatives_plus_one * theta.pow(2)
                                                     + 2 * input_delta * theta_one_minus_theta
                                                     + input_derivatives * (1 - theta).pow(2))
        logabsdet = torch.log(derivative_numerator) - 2 * torch.log(denominator)

        return outputs, logabsdet

In [153]:
o = unconstrained_rational_quadratic_spline_tf(x1,unnormalized_widths, unnormalized_heights, unnormalized_derivatives,
                                              inverse=True)

In [154]:
o_ = unconstrained_rational_quadratic_spline(x1_pt,
                                               unnormalized_widths_pt, 
                                               unnormalized_heights_pt, 
                                               unnormalized_derivatives_pt,
                                            inverse=True)

In [155]:
o[1].numpy()

array([-0.05678736, -0.0098166 ,  0.01044229,  0.0735354 , -0.13982725,
       -0.03779516, -0.0739321 , -0.03838192, -0.059025  , -0.15811585,
        0.2956113 , -0.1489464 , -0.09486258,  0.13871124, -0.1059899 ,
        0.10900342,  0.00734042, -0.09219503, -0.00995589,  0.06337629,
        0.20660332,  0.204175  , -0.12512851,  0.072561  , -0.00676407,
       -0.0847249 , -0.16315411, -0.15471725,  0.20482239, -0.05685115,
        0.10210255, -0.08964671, -0.15427431, -0.15324904,  0.09378422,
       -0.01161253,  0.17617188, -0.13418545, -0.11416645,  0.27521446,
       -0.1041963 , -0.01744099, -0.09826891, -0.16538204, -0.1498066 ,
       -0.13355224,  0.21563107,  0.05228442,  0.32177058,  0.1965512 ,
       -0.08011378, -0.14907351, -0.16229387, -0.08457458, -0.14544663,
       -0.14124686, -0.14556038,  0.10976223, -0.15190524,  0.253829  ,
       -0.08667484, -0.13336745, -0.10382208, -0.04184897, -0.15833366,
        0.04673608, -0.16403833, -0.12818797, -0.10544764,  0.01

In [156]:
o_[1].numpy()

array([-0.05678749, -0.0098166 ,  0.01044229,  0.07353435, -0.13982715,
       -0.03779517, -0.07393204, -0.03838255, -0.05902572, -0.15811591,
        0.29561293, -0.1489464 , -0.09486258,  0.13871126, -0.1059899 ,
        0.10900281,  0.00734036, -0.09219503, -0.00995589,  0.06337596,
        0.20660332,  0.20417497, -0.1251285 ,  0.07256101, -0.00676407,
       -0.08472489, -0.16315414, -0.15471706,  0.20482239, -0.05685115,
        0.10210252, -0.08964673, -0.15427394, -0.15324941,  0.09378422,
       -0.01161253,  0.17617188, -0.1341853 , -0.11416645,  0.27521303,
       -0.10419588, -0.01744106, -0.09826925, -0.16538204, -0.1498064 ,
       -0.13355221,  0.21563256,  0.05228443,  0.32177058,  0.1965512 ,
       -0.08011378, -0.14907353, -0.16229387, -0.08457451, -0.14544664,
       -0.14124684, -0.14556038,  0.10976223, -0.15190522,  0.253829  ,
       -0.08667539, -0.13336746, -0.10382172, -0.04184894, -0.15833366,
        0.04673606, -0.16403833, -0.12818809, -0.10544764,  0.01